In [1]:
#Import required packages
import pandas as pd
import numpy as np
import geopandas as gpd
import json
import requests
from functools import reduce

In [2]:
## Helper Functions to convert USGS NAS Records into geodataframe.
def _get_col_rename(df, dftype):
    """Returns a dictionary of columns to rename based on the dataframe and type('csv' or 'api')"""
    
    # Build a dictionary of column renamings for use in pandas rename function
    renamed_columns = {}
    column_names = list(df.columns)
    lower_columns = [name.lower().replace(' ','').replace('_','') for name in column_names]
    for i in range(len(column_names)):
        renamed_columns[column_names[i]] = lower_columns[i]

    if dftype == 'csv':
        # build csv rename dictionary
        renamed_columns['museumcatno'] = 'museumcatnumber'
        renamed_columns['huc8number']  = 'huc8'
    elif dftype == 'api':
        # build api rename dictionary
        renamed_columns['key']              = 'specimennumber'
        renamed_columns['decimallatitude']  = 'latitude'
        renamed_columns['decimallongitude'] = 'longitude'
        renamed_columns['latlongsource']    = 'source'
        renamed_columns['latlongaccuracy']  = 'accuracy'
    else:
        raise ValueError(f"Dataframe type '{dftype}' invalid - Accepted inputs are 'csv' or 'api'")

    return renamed_columns

In [3]:
## Helper Functions to convert USGS NAS Records into geodataframe.
def _manage_cols(df, drop_list=[], name_dict={}):
    """Private method for dropping and renaming columns in a dataframe, as well as creating one standard table from two different forms."""

    for colname in drop_list:
        if colname not in df:
            raise ValueError(f"Can't drop column '{colname}' - '{colname}' does not exist in dataframe")
    for colname in list(name_dict.keys()):
        if colname not in df:
            raise ValueError(f"Can't rename '{colname}' to '{name_dict[colname]}' - '{colname}' does not exist in dataframe")
        if colname in drop_list:
            raise ValueError(f"Can't rename '{colname}' to '{name_dict[colname]}' - '{colname}' in drop_list")

    column_names = np.setdiff1d(list(df.columns), list(name_dict.keys()))
    lower_columns = [name.lower().replace(' ','').replace('_','') for name in column_names]
    for i in range(len(column_names)):
        name_dict[column_names[i]] = lower_columns[i]
    
    df = df.drop(drop_list, axis=1)
    df = df.rename(columns=name_dict)
    
    return df

In [4]:
## Helper Functions to convert USGS NAS Records into geodataframe.
URL_BASE = 'http://nas.er.usgs.gov/api/v2/'


def api_df(species_id, limit, api_key):
    """Returns a pandas dataframe containing records about a species from the NAS database using their API"""
    
    # Check for API key
    if api_key is not None:
        url_request = f"{URL_BASE}/occurrence/search?species_ID={species_id}&api_key={api_key}"
    else:
        url_request = f"{URL_BASE}/occurrence/search?species_ID={species_id}"
    
    # Get dataframe from API request
    request_json = requests.get(url_request, params={'limit':limit}).json()
    api_df = pd.json_normalize(request_json, 'results')
    api_df = _manage_cols(api_df)

    # Add columns that are in a CSV dataframe but not an API dataframe
    api_df['country']      = np.nan
    api_df['drainagename'] = np.nan

    # Rename columns
    renamed_columns = _get_col_rename(api_df, 'api')
    api_df = api_df.rename(columns=renamed_columns)

    # Reorder columns
    cols = list(api_df.columns)
    cols = cols[0:8] + cols[33:34] + cols[8:33] + cols[34:] # country
    cols = cols[0:16] + cols[34:] + cols[16:34] # drainagename
    api_df = api_df[cols]
    
    return api_df

In [5]:
# Run API function to get records
# Visit https://nas.er.usgs.gov/api/v2/species to get your taxa species key information
zm = api_df(species_id = 5, limit = 10000, api_key = {"speciesID":5,"itis_tsn":81339,"group":"Mollusks-Bivalves","family":"Dreissenidae","genus":"Dreissena","species":"polymorpha","subspecies":"","variety":"","authority":"(Pallas, 1771)","common_name":"zebra mussel","native_exotic":"Exotic","Fresh/Marine/Brackish":"Freshwater"})

In [6]:
# Get only columns we want 
my_data = zm[["commonname", "state", "latitude", "longitude", "year", "status", "accuracy"]]

In [7]:
# Filter data 
my_data_fltr = my_data[(my_data['status'] == 'established') & (my_data['accuracy'] == 'Accurate')
& (my_data['state'] == 'Minnesota')]  

In [8]:
# Add column for Present and set values for all rows to 1 
my_data_fltr.loc[:,'Present'] = 1

C:\Users\leifh\AppData\Local\Temp\ipykernel_19000\698085495.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_data_fltr.loc[:,'Present'] = 1


In [9]:
# Check your dataset
my_data_fltr

,commonname,state,latitude,longitude,year,status,accuracy,Present
23,zebra mussel,Minnesota,46.746272,-92.124095,1989,established,Accurate,1
339,zebra mussel,Minnesota,43.993271,-91.443062,1992,established,Accurate,1
340,zebra mussel,Minnesota,44.098076,-91.707748,1992,established,Accurate,1
345,zebra mussel,Minnesota,44.160145,-91.810341,1992,established,Accurate,1
357,zebra mussel,Minnesota,44.610962,-92.610870,1992,established,Accurate,1
...,...,...,...,...,...,...,...,...
8097,zebra mussel,Minnesota,44.702140,-93.473200,2022,established,Accurate,1
8099,zebra mussel,Minnesota,46.451390,-95.552640,2022,established,Accurate,1
8100,zebra mussel,Minnesota,45.203610,-95.034590,2022,established,Accurate,1
8101,zebra mussel,Minnesota,46.646680,-94.230750,2022,established,Accurate,1


In [10]:
# export your occurence records to your project data folder as a CSV
my_data_fltr.to_csv("./data/my_data.csv")